# 総合添削問題

***

#### 問題

- 空欄を埋めてsearcherクラスを完成させてください。
- searchクラスを利用して、program study で検索した結果推薦度の高い10のURLを表示してください。
- より柔軟に対応できるように以下の要件を満たすようにsearcherクラスを変更してください。


- get_match_rows関数でwordlistにない単語が入力されるとエラーを出力してしまいます。   
try,exceptメソッドを利用することでエラーではなく　`探しているページは見つかりませんでした。`と出力するようにしてください。   


- distancescore関数はクエリに入力したときに隣接した単語間の距離の和を計算しています。   
    これをクエリのすべての単語間の距離の和を計算するようにしてください。   
    

- 現在のsearcherクラスは入力されたクエリでwordlistに存在しないものは無視されます。   
すべてのクエリの含まれるURLのみを評価対象にしてください。   
含まない単語が存在するとき`含まなかった単語を含むページはありません`と表示してください。   
またorを使うことで、例えば a or b c であればaまたはbとcが含まれたページを評価対象となるようにしてください。   


In [ ]:
import sqlite3

# 空欄を埋めてsearcher関数を完成させてください。
class searcher:
    def __init__(self,dbname):
        self.conn = sqlite3.connect(dbname)
        
    def __del__(self):
        self.conn.close()
    
    def get_match_rows(self,q):
        columnlist = 'w0.url'
        tablelist = ''
        conditionlist = ''
        words_list = []

        words = q.split(' ')
        tablenumber = 0
        for word in words:
            wordrow = self.conn.execute("select word from wordlist where word='%s'" %word).fetchone()
            if wordrow :
                words_list.append(word)
                if tablenumber > 0:
                    tablelist += ','
                    conditionlist += ' and w%d.url=w%d.url and ' % (tablenumber-1,tablenumber)
                columnlist += ',w%d.location' % tablenumber
                tablelist += 'wordlocation w%d' % tablenumber
                conditionlist+="w%d.word='%s'" % (tablenumber,word) 
                tablenumber += 1


        sql="select %s from %s where %s" % (columnlist,tablelist,conditionlist)
        cur=self.conn.execute(sql)
        rows=[row for row in cur]
        return rows,words_list

    def normalizescores(self, scores, small_flag=0):
        vsmall = 0.00001
        if small_flag:
            minscore = min(scores.values())
            return dict([(url,float(minscore)/max(vsmall,l)) for (url,l) in scores.items()])
        else:
            # 値が大きいほうが１となる正規化関数を完成させてください。


    def frequeryscore(self, rows):
        counts = dict([(row[0], 0) for row in rows])
        for row in rows:
            counts[row[0]] += 1
        return self.normalizescores(counts)

    def locationscore(self, rows):
        locations = dict([(row[0],1000000) for row in rows])
        for row in rows:
            location = sum(row[1:])
            if location < locations[row[0]]:
                locations[row[0]] = location
            return self.normalizescores(locations, small_flag=1)
    
    def distancescore(self,rows):
        if len(rows[0]) <= 2:
            return dict([(row[0],1.0) for row in rows])
        mindistance = dict([(row[0],1000000) for row in rows])
        for row in rows:
            dist = sum([abs(row[i] - row[i-1]) for i in range(2,len(row))])
            if dist < mindistance[row[0]]:
                mindistance[row[0]] = dist
        return self.normalizescores(mindistance, small_flag=1)

    
    def pagerankscore(self,rows):
        pagerank = dict([(row[0], self.conn.execute("select score from pagerank where url = '%s'" % row[0]).fetchone()[0]) for row in rows])
        return self.normalizescores(pagerank)

    def weightlist(self,rows):
        totalscores = dict([(row[0],0) for row in rows])
        # 評価関数を設定してください。
        # 評価関数の重みは、frequeryscore : 1 , locationscore : 2 , distancescore : 2 ,pagerankscore :3 としてください。
        weights = []

        for (weight, scores) in weights:
            for url in totalscores:
                totalscores[url] += weight * scores[url]
        return totalscores



    def query(self, q):
        rows,words = self.get_match_rows(q)
        scores = self.weightlist(rows)
        rankedscores = sorted([(score, url) for (url,score) in scores.items()], reverse = 1)
        recommend_list = []
        for i,(score, url) in enumerate(rankedscores[0:10]):
            recommend_list.append((url,score))
        return recommend_list

# program studyで検索してください。


#### ヒント

- リストからリストの要素のみを抜き出したいときはextendメソッドが便利です。   
リスト1.extend(リスト2)でリスト２の要素をリスト1に付け加えられます。

## 解答例

### 講座を終えてのアンケート

添削課題の提出は以下のアドレスから提出いただきますようお願いします。<br>

https://goo.gl/forms/fW7CAspZMwHuWuqk2<br><br>
以下のアドレスからアンケートにご協力頂きたく存じます。<br>
ご回答のほど、よろしくお願いいたします。

https://goo.gl/forms/WHjJQYeodIndRvyz2